## 엑셀 병합
선택한 폴더 안에 있는 모든 엑셀 파일의 칼럼 명이 같다면 병합 진행

In [2]:
import os
import pandas as pd
from tkinter import Tk, filedialog

# 현재 스크립트 상위 폴더 경로
script_dir = os.getcwd()
parent_dir = os.path.dirname(script_dir)
data_dir = os.path.join(parent_dir, 'data')
raw_dir = os.path.join(data_dir, 'raw')
merged_dir = os.path.join(raw_dir, 'agromarket_merged')

# Tkinter GUI 비활성화
root = Tk()
root.withdraw()  # Tkinter GUI 창 숨기기
root.attributes('-topmost', True)  # Tk 창을 맨 위로

# 폴더 선택 다이얼로그를 raw_dir로 기본 설정
folder_path = filedialog.askdirectory(title="폴더를 선택하세요", initialdir=raw_dir)

# 폴더 안의 모든 .xls, .xlsx 파일 리스트
excel_files = [f for f in os.listdir(folder_path) 
               if f.endswith('.xls') or f.endswith('.xlsx')]

all_dfs = []
columns_reference = None  # 모든 파일의 칼럼명이 같은지 비교하기 위한 레퍼런스

for excel_file in excel_files:
    file_path = os.path.join(folder_path, excel_file)
    df = pd.read_excel(file_path)
    
    # 첫 번째 파일의 칼럼을 기준으로 삼음
    if columns_reference is None:
        columns_reference = list(df.columns)
    else:
        # 칼럼이 동일한지 검사
        if list(df.columns) != columns_reference:
            print(f"[주의] 칼럼 불일치 발생: {excel_file}")
            print(f"기준 칼럼   : {columns_reference}")
            print(f"현재 파일 칼럼: {list(df.columns)}\n")
            raise ValueError("칼럼명이 일치하지 않아 병합할 수 없습니다.")
    
    # 확인 후 문제 없으면 병합 리스트에 추가
    all_dfs.append(df)

# 문제없이 읽힌 모든 DataFrame 병합
merged_df = pd.concat(all_dfs, ignore_index=True)

merged_csv_path = os.path.join(merged_dir, 'merged_agromarket.csv')
merged_df.to_csv(merged_csv_path)

print("선택한 폴더:", folder_path)
print("엑셀 파일 리스트:", excel_files)
print("병합된 데이터프레임 크기:", merged_df.shape)


선택한 폴더: C:/projects/ClimateMarketForecast/data/raw/agromarket_merged
엑셀 파일 리스트: ['merged_agromarket_2019_Q1.xlsx', 'merged_agromarket_2019_Q2.xlsx', 'merged_agromarket_2019_Q3.xlsx', 'merged_agromarket_2019_Q4.xlsx', 'merged_agromarket_2020_Q1.xlsx', 'merged_agromarket_2020_Q2.xlsx', 'merged_agromarket_2020_Q3.xlsx', 'merged_agromarket_2020_Q4.xlsx', 'merged_agromarket_2021_Q1.xlsx', 'merged_agromarket_2021_Q2.xlsx', 'merged_agromarket_2021_Q3.xlsx', 'merged_agromarket_2021_Q4.xlsx', 'merged_agromarket_2022_Q1.xlsx', 'merged_agromarket_2022_Q2.xlsx', 'merged_agromarket_2022_Q3.xlsx', 'merged_agromarket_2022_Q4.xlsx', 'merged_agromarket_2023_Q1.xlsx', 'merged_agromarket_2023_Q2.xlsx', 'merged_agromarket_2023_Q3.xlsx', 'merged_agromarket_2023_Q4.xlsx', 'merged_agromarket_2024_Q1.xlsx', 'merged_agromarket_2024_Q2.xlsx', 'merged_agromarket_2024_Q3.xlsx', 'merged_agromarket_2024_Q4.xlsx']
병합된 데이터프레임 크기: (15305393, 8)


## 전처리
- 법인 칼럼 제거
- 총 물량 상위 30개 품목만 필터링
- 0 이하 데이터 제거
- '부류', '품목' 칼럼의 결측치 데이터 삭제
- csv로 저장

In [12]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv('../data/raw/agromarket_merged/merged_agromarket.csv', index_col=False)

# 1) '법인' 칼럼 drop
df.drop(columns=['법인'], inplace=True)

# 2) '품목' 별 '물량(kg)' 상위 30개 품목만 필터링
#    - 품목별 총 물량을 합산한 다음 상위 30개 품목만 추출
top30_items = (df.groupby('품목')['물량(kg)']
               .sum()
               .sort_values(ascending=False)
               .head(30)
               .index
              )
df = df[df['품목'].isin(top30_items)]

# 3) 0 이하 데이터(0과 음수 값) 제거
df = df[(df['물량(kg)'] > 0) & (df['금액(원)'] > 0)]

# 4) '부류', '품목' 칼럼의 결측치 데이터 삭제
df.dropna(subset=['부류', '품목'], inplace=True)

# CSV로 저장
df.to_csv('../data/processed/preprocessed_agromarket.csv')

## DB 저장